In [5]:
import pandas as pd 
import os 
import re

In [6]:
# Get RAW directory from notebook and snakemake .. 
RAW_DIRECTORY = os.getcwd().replace("parsers","") + "/raw"

RAW_FILE = f"{RAW_DIRECTORY}/genemap2.txt"
GENE_MAP =  f"{RAW_DIRECTORY}/gene_map.csv"

In [7]:
# Create a genemap :   entrez_id : ensembl_id 
genemap = pd.read_csv(GENE_MAP, sep="\t")
genemap = genemap[["entrez_id","ensembl_gene_id"]].drop_duplicates().set_index("entrez_id").to_dict()["ensembl_gene_id"]

In [ ]:
# 10% du fichier OMIM que j'ai n'a pas de ENST 
# 6% entrez.. 

In [8]:
def extract_info(phenos):
   # phenos ="'{Bronchiectasis with or without elevated sweat chloride 1, modifier of}, 211400 (3), Autosomal dominant; Sweat chloride elevation without CF (3); {Hypertrypsinemia, neonatal} (3); Cystic fibrosis, 219700 (3), Autosomal recessive; Congenital bilateral absence of vas deferens, 277180 (3), Autosomal recessive; {Pancreatitis, hereditary}, 167800 (3), Autosomal dominant'"

    inheritances = set()
    diseases = set()
    for pheno in phenos.split(";"):

        # search inheritance 
        inheritance = re.findall("\(\d+\),([\w\s]+)", pheno)
        if inheritance:
            inheritances.add(inheritance[0].strip())

        # search diseases 
        disease = re.findall("(.+)\(\d+\)", pheno)
        if disease:
            diseases.add(re.sub("[{\(\)}\?]", "", disease[0]))

    return "|".join(inheritances), "|".join(diseases)
    
    

In [34]:
# TODO : get version of file ... 

with open(RAW_FILE) as file:
    headers = [next(file) for i in range(4)][-1].replace("#","").strip().split("\t")
    

df = pd.read_csv(RAW_FILE,sep="\t", comment="#", names=headers)

df = df[["Entrez Gene ID","Phenotypes", "MIM Number"]].dropna()
df.columns = ["entrez","phenotype", "omim_id"]

df["entrez"] = df["entrez"].astype(int)

#df["ensembl_id"] = df["entrez"].map(genemap)

df["inheritance"] = df["phenotype"].apply(lambda x: extract_info(x)[0])
df["disease"] = df["phenotype"].apply(lambda x: extract_info(x)[1])


df["ensembl_id"] = df["entrez"].map(genemap)
df = df[["ensembl_id", "inheritance", "disease","omim_id"]].dropna()
df

,ensembl_id,inheritance,disease,omim_id
22,ENSG00000187608,Autosomal recessive,"Immunodeficiency 38, 616126",147571
23,ENSG00000188157,Autosomal recessive,"Myasthenic syndrome, congenital, 8, with pre- ...",103320
28,ENSG00000186827,Autosomal recessive,"Immunodeficiency 16, 615593",600315
30,ENSG00000176022,Autosomal recessive,"Ehlers-Danlos syndrome, spondylodysplastic ty...",615291
36,ENSG00000107404,Autosomal dominant,"Robinow syndrome, autosomal dominant 2, 616331",601365
...,...,...,...,...
17131,ENSG00000185973,X,"Autism, susceptibility to, X-linked 6, 300872",300777
17145,ENSG00000185960,Pseudoautosomal dominant|Pseudoautosomal reces...,"Short stature, idiopathic familial, 300582 | ...",400020
17154,ENSG00000184895,Y|X,"46XY sex reversal 1, 400044 | 46XX sex reversa...",480000
17160,ENSG00000092377,Y,"Deafness, Y-linked 2, 400047",400033


In [83]:
output = snakemake.output[0]

descriptions = (
("title", "omim"),
("description", "omim"),
("version", "no idea"),
("ensembl_id", "ensembl gene id "),
("inheritance", "inheritance "),
("disease", "diseases "),
)

with open(output,"w") as f:
    for d in descriptions:
        f.write("#" + "\t".join(d) +"\n")
        
    df.to_csv(f,sep="\t", index=False)

['{Alkaline phosphatase, plasma level of, QTL 2}, 612367 (2)',
 '{Anorexia nervosa, susceptibility to, 1}, 606788 (2)',
 '{Basal cell carcinoma, susceptibility to, 1}, 605462 (2)',
 '[Bone mineral density QTL 3], 606928 (2)',
 '{Melanoma, cutaneous malignant, 1}, 155600 (2), Autosomal dominant',
 'Cataract 8, multiple types, 115665 (2), Autosomal dominant',
 '{Dyslexia, susceptibility to, 8}, 608995 (2), Multifactorial, Autosomal dominant',
 '{Inflammatory bowel disease 7}, 605225 (2)',
 '{Multiple sclerosis, susceptibility to, 4}, 612596 (2)',
 'Myopia 14, 610320 (2), Autosomal dominant',
 '{Psoriasis susceptibility 7}, 605606 (2)',
 'Immunodeficiency 38, 616126 (3), Autosomal recessive',
 'Myasthenic syndrome, congenital, 8, with pre- and postsynaptic defects, 615120 (3), Autosomal recessive',
 '?Immunodeficiency 16, 615593 (3), Autosomal recessive',
 'Al-Gazali syndrome, 609465 (3); Ehlers-Danlos syndrome, spondylodysplastic type, 2, 615349 (3), Autosomal recessive; Spondyloepimetap

,Chromosome,Genomic Position Start,Genomic Position End,Cyto Location,Computed Cyto Location,MIM Number,Gene Symbols,Gene Name,Approved Symbol,Entrez Gene ID,Ensembl Gene ID,Comments,Phenotypes,Mouse Gene Symbol/ID
6764,chr7,117480024,117668664,7q31.2,7q31.2,602421,"CFTR, ABCC7, CF, MRP7",Cystic fibrosis transmembrane conductance regu...,CFTR,1080.0,ENSG00000001626,distal and 5' to MET,{Bronchiectasis with or without elevated sweat...,Cftr (MGI:88388)
